# Vector Databases - Research

Vector search is a cutting-edge approach to searching and retrieving data that leverages the power of vector similarity calculations. Unlike traditional keyword-based search, which matches documents based on the occurrence of specific terms, vector search focuses on the semantic meaning and similarity of data points. By representing data as vectors in a high-dimensional space, vector search enables more accurate and intuitive search results.

In [ ]:
pip install python-dotenv==1.0.1 langchain==0.2.1 langchain-community==0.2.1 scikit-learn==1.5.0

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Load and split the data

First, we need to load a document. Let's try with a pdf and a markdown file.

Then, we will split it into smaller chunks of text. This serves several important purposes:
- Granularity: By splitting a document into smaller chunks, you can retrieve more specific and relevant pieces of information. If you work with large documents as single chunks, retrieval can become inefficient and less precise.
- Search Performance: Smaller chunks improve the performance of search algorithms. It's easier to match a query against smaller, more focused pieces of text rather than a large document.
- Computational efficiency: Working with entire documents as single units can be memory-expensive and slow. Splitting documents into chunks allows for more efficient use of memory and computational resources.

Practical example: Imagine we have a large document, such as a product manual or a scientific paper. If a user asks a question like "How do I reset the device?" or "What is the conclusion of the study?", splitting the document into smaller chunks allows the chatbot to:
- Efficiently locate and retrieve the most relevant section about device resetting instructions or the conclusion of the study.
- Avoid returning irrelevant parts of the document, which might confuse the user.
- Provide a faster response by only processing a small portion of the document.

In [ ]:
pip install pypdf==4.2.0 unstructured==0.14.7 markdown==3.6

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define how the text should be split:
#  - Each chunk should be up to 512 characters long.
#  - There should be an overlap of 64 characters between consecutive chunks. 
#  - This overlap helps maintain context across the chunks.
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [4]:
# PDF

from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("example_data/example_pdf.pdf")

# Load pdf and split into chunks.
pdf_chunks = pdf_loader.load_and_split(text_splitter=splitter)

# Get the number of chunks
print(f"Number of chunks: {len(pdf_chunks)}")

# Print the first 5 chunks
for chunk in pdf_chunks[:5]:
    print(chunk)

Number of chunks: 311
page_content='GENERATIVE AI: HYPE,  OR \nTRUL\nY TRANSFORMATIVE?ISSUE 120 | July 5, 2023 | 12:28 PM EDT”\x01\x01\x01\x01\x01P\x01\x01\x01\x01\x01\x01\x01\x01\x01 \x01\x01 \x01 \x01Global Macro  \nResearch\nInvestors should consider this report as only a single factor in making their investment decision. For \nReg AC certiﬁcation and other important disclosures, see the Disclosure Appendix, or go to www.gs.com/research/hedge.html.\nThe Goldman Sachs Group, Inc.Since the release of OpenAI’s generative AI tool ChatGPT in November, investor' metadata={'source': 'example_data/example_pdf.pdf', 'page': 0}
page_content='interest in generative AI technology has surged. The disruptive potential of  this technology, and whether the hype around it—and market pricing—has gone too far, is Top of Mind. We speak with Conviction’s Sarah Guo, NYU’s Gary Marcus, and GS GIR’s US software and internet analysts Kash Rangan and Eric Sheridan about what the technology can—and can’t—do a

In [5]:
# Markdown

from langchain_community.document_loaders import UnstructuredMarkdownLoader

md_loader = UnstructuredMarkdownLoader("example_data/example_markdown.md")

# Load pdf and split into chunks.
md_chunks = md_loader.load_and_split(text_splitter=splitter)

# Get the number of chunks
print(f"Number of chunks: {len(md_chunks)}")

# Print the first 5 chunks
for chunk in md_chunks[:5]:
    print(chunk)

Number of chunks: 7
page_content='The Football World Cup\n\nThe FIFA World Cup, often simply referred to as the World Cup, is the premier international football competition. Organized by the Fédération Internationale de Football Association (FIFA), it features national teams from around the globe competing for the most coveted trophy in the sport. The tournament is held every four years and is one of the most widely viewed and followed sporting events in the world.\n\nHistory' metadata={'source': 'example_data/example_markdown.md'}
page_content='History\n\nThe first World Cup was held in 1930 in Uruguay, with the host nation emerging as the champions. Since then, the tournament has grown significantly in size and popularity. Originally featuring just 13 teams, the World Cup now includes 32 teams in the final tournament, with plans to expand to 48 teams in the near future.\n\nFormat\n\nThe World Cup is divided into two main stages:' metadata={'source': 'example_data/example_markdown.md'

## Embeddings

Now we will proceed to transform our text junks into vector embeddings.

Vector embeddings is a powerful technique for transforming complex data into numerical forms that can be easily processed and analyzed by machine learning algorithms. It basically allows us to take virtually any data type and represent it as vectors.

But it isn't as simple as just turning data into vectors. We want to ensure that we can perform tasks on this transformed data without losing the data's original meaning. For example, if we want to compare two sentences, we don't want just to compare the words they contain but rather whether or not they mean the same thing. 

To preserve the data's meaning, we need to understand how to produce vectors where relationships between the vectors make sense. To do this, we need what's known as an embedding model. We apply a pre-trained machine learning model that will produce a representation of this data that is more compact while preserving what's meaningful about the data.

The goal of embeddings is to capture the semantic meaning or relationships between data points in a way that similar items are close together in the vector space, and dissimilar items are far apart. For example, consider two words "king" and "queen". An embedding might map these words to vectors such that the difference between the "king" and "queen" vectors is similar to the difference between the "man" and "woman" vectors. This reflects the underlying semantic relationships.

Key characteristics of embeddings:
- Dimensionality: The number of elements in the vector. Higher dimensions can capture more complex relationships but are computationally more expensive.
- Similarity: Measured using metrics like cosine similarity or euclidean distance, which help in finding how close or far two vectors are from each other.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
# Calculate cosine similarity between vectors.
# Values close to 1 indicate very similar vectors, while values close to 0 indicate very different vectors.
# Cosine similarity is particularly useful when the magnitude of the vectors is not as important as their direction. 
# It is often used in text analysis and information retrieval where the orientation (semantic meaning) of vectors 
# matters more than their magnitude.
def compare_embeddings(vector_1, vector_2):
    similarity = cosine_similarity([vector_1], [vector_2])
    print(f"Cosine similarity: {similarity[0][0]}")
    

In [37]:
# Some sentences to test different embedding models
sentences = [
    # Include the same words but have different semantics. Similarity should be low.
    ["man pets dog", "dog bites man"],      
    ["today is Monday", "today is Friday"],
    # Have related meanings but no common words. Similarity should be high.
    ["I need a new phone", "my old device is broken"], 
    ["tourism numbers are collapsing", "travel industry fears Covid-19 crisis will cause more companies to enter bankruptcy"],
    # Completely unrelated in terms of their content and context. Similarity should be low.
    ["the stock market experienced a significant decline last week", "she enjoys painting landscapes in her free time"]
]

In [35]:
# Function to test an embedding model applying it to a set of sentences and analyzing similarity
def test_embeddings(model):
    for sentences_pair in sentences:
        vector_1 = model.embed_query(sentences_pair[0])
        print(f"{sentences_pair[0]}:")
        print(f" Dimensionality: {len(vector_1)}")
        print(f" Sample: {vector_1[:5]}")
        print()
        
        vector_2 = model.embed_query(sentences_pair[1])
        print(f"{sentences_pair[1]}:")
        print(f" Dimensionality: {len(vector_2)}")
        print(f" Sample: {vector_2[:5]}")
        print()

        compare_embeddings(vector_1, vector_2)
        print("---")

#### Google AI

In [ ]:
pip install langchain-google-genai

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [38]:
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

test_embeddings(google_embeddings)

man pets dog:
 Dimensionality: 768
 Sample: [0.03934462368488312, 0.019560974091291428, -0.023450257256627083, -0.004843460395932198, -0.01836157590150833]

dog bites man:
 Dimensionality: 768
 Sample: [0.045345816761255264, 0.0035485224798321724, 0.015394225716590881, -0.030298084020614624, -0.00495710177347064]

Cosine similarity: 0.8259540237192756
---
today is Monday:
 Dimensionality: 768
 Sample: [0.0022789041977375746, -0.015724509954452515, 0.0013650263426825404, -0.03787536919116974, 0.023058727383613586]

today is Friday:
 Dimensionality: 768
 Sample: [0.009477449581027031, -0.010321601293981075, -0.01116954442113638, -0.051544252783060074, 0.014089941047132015]

Cosine similarity: 0.9181408679004186
---
I need a new phone:
 Dimensionality: 768
 Sample: [0.02577350102365017, 0.0053873201832175255, -0.053824517875909805, -0.03254551440477371, 0.051140397787094116]

my old device is broken:
 Dimensionality: 768
 Sample: [0.01551913470029831, 0.022993290796875954, -0.045324489474

#### Hugging Face

In [ ]:
pip install langchain_huggingface==0.0.3 huggingface_hub==0.23.4

In [63]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

In [64]:
hface_embeddings = HuggingFaceEndpointEmbeddings()

test_embeddings(hface_embeddings)

man pets dog:
 Dimensionality: 768
 Sample: [0.02580537460744381, -0.022475017234683037, -0.014024853706359863, -0.005074481945484877, 0.005956384819000959]

dog bites man:
 Dimensionality: 768
 Sample: [0.06021416559815407, -0.0734042078256607, -0.00042923304135911167, -0.00997690949589014, 0.03659734129905701]

Cosine similarity: 0.5876407567728386
---
today is Monday:
 Dimensionality: 768
 Sample: [0.007682239171117544, -8.812590385787189e-05, -0.013025548309087753, 0.04075957089662552, -0.06397198885679245]

today is Friday:
 Dimensionality: 768
 Sample: [-0.008920947089791298, 0.038364000618457794, -0.0027555814012885094, 0.03241955488920212, -0.05083586648106575]

Cosine similarity: 0.8555735624527724
---
I need a new phone:
 Dimensionality: 768
 Sample: [0.0016399134183302522, 0.05188266932964325, 0.003045764984562993, 0.007006904575973749, 0.01983613707125187]

my old device is broken:
 Dimensionality: 768
 Sample: [0.020737426355481148, 0.04476892948150635, 0.04422058537602424

Embedding model | Sentence 1 | Sentence 2 | Cosine similarity | Expected result
:---: | :---: | :---: | :---: | :---:
Google | "man pets dog" | "dog bites man" | 0.83 | ❌
Google | "today is Monday" | "today is Friday" | 0.92 | ❌
Google | "I need a new phone" | "my old device is broken" | 0.77 | ✅
Google | "tourism numbers are collapsing" | "travel industry fears Covid-19 crisis will cause more companies to enter bankruptcy" | 0.73 | ✅
Google | "the stock market experienced a significant decline last week" | "she enjoys painting landscapes in her free time" | 0.92 | ❌
Hugging Face | "man pets dog" | "dog bites man" | 0.59 | ✅
Hugging Face | "today is Monday" | "today is Friday" | 0.85 | ❌
Hugging Face | "I need a new phone" | "my old device is broken" | 0.51 | ❌
Hugging Face | "tourism numbers are collapsing" | "travel industry fears Covid-19 crisis will cause more companies to enter bankruptcy" | 0.53 | ❌
Hugging Face | "the stock market experienced a significant decline last week" | "she enjoys painting landscapes in her free time" | -0.1 | ✅



# Astra DB

For using Datastax Astra DB:
- Create a database in https://astra.datastax.com/
- Obtain your database API endpoint, located under Database Details > API Endpoint, and save it as an environment variable called: ASTRA_DB_API_ENDPOINT
- Generate a token and save it as an environment variable called: ASTRA_DB_APPLICATION_TOKEN

In [ ]:
pip install langchain-astradb==0.3.3

In [17]:
from langchain_astradb import AstraDBVectorStore

In [19]:
vstore = AstraDBVectorStore(
    embedding=embe,
    collection_name="astra_vector_db",
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
)

ValueError: Either an `embedding` or a `collection_vector_service_options`                    must be provided.